In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import random
from tqdm import tqdm

In [4]:
filename = 'test.txt'
with open(filename, 'r', encoding='utf-8') as f:
    text = f.read().lower()

print(len(text))

1197


In [5]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
vocab_size = len(chars)

maxlen = 40
step = 3     
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print(len(sentences))

X = np.zeros((len(sentences), maxlen, vocab_size), dtype=bool)
y = np.zeros((len(sentences), vocab_size), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

386


In [6]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, vocab_size)))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

C:\Users\Firo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-7) / temperature 
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length=400, temperature=0.5):
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    result_text = sentence
    
    for i in range(length):
        x_pred = np.zeros((1, maxlen, vocab_size))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char
        result_text += next_char

    return result_text

In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-7) / temperature 
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length=400, temperature=0.5):
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    result_text = sentence
    
    for i in range(length):
        x_pred = np.zeros((1, maxlen, vocab_size))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char
        result_text += next_char

    return result_text

  0%|          | 0/25 [00:00<?, ?it/s]

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 3.7957


  4%|▍         | 1/25 [00:00<00:18,  1.30it/s]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 3.6761


  8%|▊         | 2/25 [00:00<00:09,  2.49it/s]

...

 92%|█████████▏| 23/25 [00:04<00:00,  6.36it/s]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.7840


 96%|█████████▌| 24/25 [00:04<00:00,  6.34it/s]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 2.6592


100%|██████████| 25/25 [00:04<00:00,  5.70it/s]


In [9]:
final_text = generate_text(length=500, temperature=0.7)
print(final_text)

о превратностях рынка ценных бумаг и игн и ееновапо вашла ие ко во пеедеяеиро уорслдееногсе сбы ке сене с коюие петснскенсое сге пеекокенехоиыа ег  ы о ви исв исецесо»еелх него не ен еге ек леь наег сделыжеллы аелеесум и иоленинесесьсииеелдонеадо нелеогее бделрееуидегьл аые огеле гелм, иенеа е сид восдокииеленсеее галоен вабы веа»нсмь ссовакояситосиделсе не ега и.егосесеесее ипо ь аиреы »е едеелиеес рел  меске  есгоиикекааеогодеесине по сьена ре ы ееалгрееологоеа
не ны с мерь делиоссеинеелвоолене не ьо маоовса иее)не етго в  ирелппга 


In [ ]:
final_text = generate_text(length=500, temperature=0.7)
print(final_text)